## Streaming app from stocks

In [1]:
import requests
from datetime import datetime
from pixiedust.display.app import *
from pixiedust.display.streaming import *

class StockStreamingAdapter(StreamingDataAdapter):
    def __init__(self):
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=demo'
        self.payload = requests.get(url).json()
        self.timeSeries = self.payload['Time Series (1min)']
        self.metadata = self.payload['Meta Data']
        self.allX = [datetime.strptime(dt, '%Y-%m-%d %H:%M:%S').strftime("%H:%M") for dt in self.timeSeries]
        self.allY = [w['1. open'] for v,w in iteritems(self.timeSeries)]
        self.ptr = None
    
    def getNextData(self):
        if self.ptr is None:
            self.ptr = 9
            return self.allY[0:10]
        self.ptr = self.ptr + 1
        return self.allY[self.ptr:self.ptr+1]

Pixiedust database opened successfully


In [2]:
from pixiedust.display.streaming.bokeh import *
@PixieApp
class StockStreamingApp():    
    def setup(self):
        self.streamingData = StockStreamingAdapter()
        self.streamingDisplay = None

    def newDisplayHandler(self, options, entity):
        if self.streamingDisplay is None:
            self.streamingDisplay = LineChartStreamingDisplay(options, entity)
            self.streamingDisplay.windowSize = 20
        return self.streamingDisplay
        
    @route()
    def main(self):
        return"""
            <div pd_entity="streamingData" pd_refresh_rate="1000">
            </div>
        """

#run the app
stockApp = StockStreamingApp()
stockApp.run(runInDialog='false')